In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import utils
import random

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# hyperparameters
batch_iterations = 100
batch_size = 32
full_iterations = 25
learning_rate = 0.01
reg_eta = 0.01

# dimensionalities
dim_lstm = 300
dim_word = 300
dim_aspect = 5
dim_sentence = 80
dim_polarity = 3

# setup utils object
isSample = True
u = utils.UTILS(batch_size, dim_sentence, dim_polarity, isSample)

In [3]:
# define tf placeholders
X = tf.placeholder(tf.int32, [None, dim_sentence])
y = tf.placeholder(tf.float32, [None, dim_polarity])
seqlen = tf.placeholder(tf.int32, [None])
lr = tf.placeholder(tf.float32, [])

In [4]:
# define tf variables
with tf.variable_scope('bilstm_vars'):
    with tf.variable_scope('weights', reuse = tf.AUTO_REUSE):
        lstm_w = tf.get_variable(
            name = 'softmax_w',
            shape = [dim_lstm * 2, dim_polarity],
            initializer = tf.random_uniform_initializer(-0.003, 0.003),
            regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
        )
    with tf.variable_scope('biases', reuse = tf.AUTO_REUSE):
        lstm_b = tf.get_variable(
            name = 'softmax_b',
            shape = [dim_polarity],
            initializer = tf.random_uniform_initializer(-0.003, 0.003),
            regularizer = tf.contrib.layers.l2_regularizer(reg_eta)
        )

Instructions for updating:
Use the retry module or similar alternatives.


In [6]:
# define lstm model
def dynamic_lstm(inputs, seqlen):
    inputs = tf.nn.dropout(inputs, keep_prob=1.0)
    with tf.name_scope('bilstm_model'):
        forward_lstm_cell = tf.contrib.rnn.LSTMCell(dim_lstm)
        backward_lstm_cell = tf.contrib.rnn.LSTMCell(dim_lstm)
        outputs, states = tf.nn.bidirectional_dynamic_rnn(
            forward_lstm_cell,
            backward_lstm_cell,
            inputs = inputs,
            sequence_length = seqlen,
            dtype = tf.float32,
            scope = 'bilstm'
        )
        forward_outputs, backward_outputs = outputs
        backward_outputs = tf.reverse_sequence(backward_outputs, tf.cast(seqlen, tf.int64), seq_dim=1)
        outputs = tf.concat([forward_outputs, backward_outputs], 2)
        size = tf.shape(outputs)[0]
        index = tf.range(0, size) * dim_sentence + seqlen - 1
        output = tf.gather(tf.reshape(outputs, [-1, dim_lstm * 2]), index)  # batch_size * n_hidden * 2
    predict = tf.matmul(output, lstm_w) + lstm_b
    return predict


In [8]:
# define operations
# tf.reset_default_graph()
pred = dynamic_lstm(tf.nn.embedding_lookup(u.gloveDict, X), seqlen)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)
correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
# batch training
test_X, test_y, test_seqlen, _ = u.getData('test')
lrval = 0.01
with tf.Session() as sess:
    sess.run(init)
    for i in range(batch_iterations):
        batch_X, batch_y, batch_seqlen, _ = u.nextBatch(batch_size)
        lrval *= (1. / (1. + 0.2 * i))
        sess.run(optimizer, feed_dict = {X: batch_X, y: batch_y, seqlen: batch_seqlen, lr: lrval})
        if i > 0 and i % 4 == 0:
            loss_train, accuracy_train = sess.run([loss, accuracy], feed_dict = {X: batch_X, y: batch_y, seqlen: batch_seqlen})
            print('step: %s, train loss: %s, train accuracy: %s' % (i, loss_train, accuracy_train))
            loss_test, accuracy_test = sess.run([loss, accuracy], feed_dict = {X: test_X, y: test_y, seqlen: test_seqlen})
            print('step: %s, test loss: %s, test accuracy: %s' % (i, loss_test, accuracy_test))

In [9]:
# full dataset training
test_X, test_y, test_seqlen, _ = u.getData('test')
train_X, train_y, train_seqlen, _ = u.getData('train')
lrval = 0.01
with tf.Session() as sess:
    sess.run(init)
    for i in range(full_iterations):
        lrval *= (1. / (1. + 0.2 * i))
        sess.run(optimizer, feed_dict = {X: train_X, y: train_y, seqlen: train_seqlen, lr: lrval})
        if i > 0 and i % 4 == 0:
            loss_train, accuracy_train = sess.run([loss, accuracy], feed_dict = {X: train_X, y: train_y, seqlen: train_seqlen, lr: 0.01})
            print('step: %s, train loss: %s, train accuracy: %s' % (i, loss_train, accuracy_train))
            loss_test, accuracy_test = sess.run([loss, accuracy], feed_dict = {X: test_X, y: test_y, seqlen: test_seqlen, lr: 0.01})
            print('step: %s, test loss: %s, test accuracy: %s' % (i, loss_test, accuracy_test))

step: 4, train loss: 0.9202549, train accuracy: 0.62938386
step: 4, test loss: 0.82367855, test accuracy: 0.7123288
step: 8, train loss: 0.9174761, train accuracy: 0.6341232
step: 8, test loss: 0.79527277, test accuracy: 0.7123288
step: 12, train loss: 0.91696906, train accuracy: 0.6341232
step: 12, test loss: 0.7949051, test accuracy: 0.7123288
step: 16, train loss: 0.91696197, train accuracy: 0.6341232
step: 16, test loss: 0.7949019, test accuracy: 0.7123288
step: 20, train loss: 0.916962, train accuracy: 0.6341232
step: 20, test loss: 0.79490197, test accuracy: 0.7123288
step: 24, train loss: 0.916962, train accuracy: 0.6341232
step: 24, test loss: 0.79490197, test accuracy: 0.7123288
